In [1]:
!pwd

/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1


In [122]:
!kfp component build src/components/data_download --component-filepattern data_download_component.py 

Building component using KFP package path: kfp==2.7.0
Found 1 component(s) in file /Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/data_download/data_download_component.py:
Dataset download: ComponentInfo(name='Dataset download', function_name='dataset_download', func=<function dataset_download at 0x108cfe830>, target_image='mohitverma1688/data-download_component:v0.4', module_path=PosixPath('/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/data_download/data_download_component.py'), component_spec=ComponentSpec(name='dataset-download', implementation=Implementation(container=ContainerSpecImplementation(image='mohitverma1688/data-download_component:v0.4', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet --no-warn-scrip

In [121]:
%%writefile src/components/data_download/data_download_component.py
from kfp import dsl

@dsl.component(base_image='python:3.10-slim',
               target_image='mohitverma1688/data-download_component:v0.4',
               packages_to_install=['pathlib','requests','kfp-kubernetes'])



def dataset_download(url: str, base_path:str, 
                     ):

    import os
    import requests
    import zipfile
    from pathlib import Path

    #Import the enviornment variable 

    #os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio-service.kubeflow:9000"
    #os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
    #os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
    
    #s3 = boto3.client(
    #    "s3",
    #    endpoint_url="http://minio-service.kubeflow:9000",
    #    aws_access_key_id=aws_access_key_id,
    #    aws_secret_access_key=aws_secret_access_key,
    #    config=Config(signature_version="s3v4"),
    #)
    # Create data bucket if it does not yet exist
    #response = s3.list_buckets()
    #input_bucket_exists = False
    #for bucket in response["Buckets"]:
    #    if bucket["Name"] == input_bucket:
    #        input_bucket_exists = True
    #        
    #if not input_bucket_exists:
    #    s3.create_bucket(ACL="public-read-write", Bucket=input_bucket)


    # Save zip files to S3 import_bucket
    data_path = Path(base_path)
    
    if data_path.is_dir():
      print(f"{data_path} directory exists.")
    else:
      print(f"Did not find {data_path} directory, creating one...")
      data_path.mkdir(parents=True,exist_ok=True)


    # Download pizza , steak and sushi data and upload to the s3 bucket. This is example code to save the downloaded data to the bucket
    with open(f"{data_path}/data.zip", "wb") as f:
        request = requests.get(f"{url}")
        print(f"Downloading data from {url}...")
        f.write(request.content)
    #    for root, dir, files in os.walk(data_path):
    #        for filename in files:
    #            local_path = os.path.join(root,filename)
    #            s3.upload_file(
    #               local_path,
    #               input_bucket,
    #               f"{local_path}",
    #               ExtraArgs={"ACL": "public-read"},
    #             )

    # unzip the data to use the data in the next step. Data will be stored in PVC.
    with zipfile.ZipFile(data_path/"data.zip", "r") as zip_ref:
      print("Unzipping data...")
      zip_ref.extractall(data_path)




Overwriting src/components/data_download/data_download_component.py


In [59]:
!kfp component build src/components/model_train_cnn --component-filepattern model_train_component.py --no-push-image

Building component using KFP package path: kfp==2.7.0
Found 1 component(s) in file /Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_train_cnn/model_train_component.py:
Model train: ComponentInfo(name='Model train', function_name='model_train', func=<function model_train at 0x106f9e830>, target_image='mohitverma1688/model_train_component:v0.24', module_path=PosixPath('/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_train_cnn/model_train_component.py'), component_spec=ComponentSpec(name='model-train', implementation=Implementation(container=ContainerSpecImplementation(image='mohitverma1688/model_train_component:v0.24', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet --no-warn-script-location \'pandas\' \'mat

In [58]:
%%writefile src/components/model_train_cnn/model_train_component.py
from kfp import dsl
from kfp import compiler
from typing import Dict
from kfp.dsl import Dataset,Output,Artifact,OutputPath,InputPath, Model,HTML

@dsl.component(base_image='mohitverma1688/model_train_component:v0.4',
               target_image='mohitverma1688/model_train_component:v0.24',
               packages_to_install=['pandas','matplotlib']
               )

def model_train(num_epochs:int, 
                batch_size:int, 
                hidden_units:int,
                learning_rate: float,
                train_dir: str,
                test_dir: str,
                model_name: str,
                model_dir: str,
                model_artifact_path: OutputPath('Model'),
                parameters_json_path: OutputPath('Artifact'),
                train_loss: Output[HTML],
                export_bucket: str = "modelbucket", 
               ) -> Dict[str, list] :

            import os
            import json
            import pandas as pd
            import torch
            import data_setup, model_train, model_builder, utils
            from io import BytesIO
            import base64
            import matplotlib.pyplot as plt

            from torchvision import transforms
         
            

            # Setup hyperparameters
            NUM_EPOCHS = num_epochs
            BATCH_SIZE = batch_size
            HIDDEN_UNITS = hidden_units
            LEARNING_RATE = learning_rate
            MODEL_NAME = model_name
            MODEL_DIR = model_dir
            EXPORT_BUCKET = export_bucket

    

            # Setup directories
            TRAIN_DIR = train_dir
            TEST_DIR = test_dir

            # Setup target device
            device = "cuda" if torch.cuda.is_available() else "cpu"

            # Create transforms
            data_transform = transforms.Compose([
              transforms.Resize((64, 64)),
              transforms.ToTensor()
            ])

            # Create DataLoaders with help from data_setup.py
            train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
                train_dir=TRAIN_DIR,
                test_dir=TEST_DIR,
                transform=data_transform,
                batch_size=BATCH_SIZE
            )

            # Create model with help from model_builder.py
            model = model_builder.TinyVGG(
                input_shape=3,
                hidden_units=HIDDEN_UNITS,
                output_shape=len(class_names)
            ).to(device)

            # Set loss and optimizer
            loss_fn = torch.nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(),
                                         lr=LEARNING_RATE)

            # Start training with help from engine.py
            result = model_train.train(model=model,
                         train_dataloader=train_dataloader,
                         loss_fn=loss_fn,
                         optimizer=optimizer,
                         epochs=NUM_EPOCHS,
                         device=device)
    
             

            df = pd.DataFrame(result)
    
            # Example Saving dataset as an artifact in kubeflow pipeline. 
            #output_dataset.path = f"{output_dataset.path}.csv"
            #df.to_csv(output_dataset.path, index=False)
            #with open(output_dataset.path, "w") as file:
            #file.write(df.to_csv(index=False))
        

            #Get number of epochs

            epochs = range(len(df))
        

            # Plot train loss in the kfp pipeline for visialization. 
            tmpfile = BytesIO()
            plt.figure(figsize=(15,10))
            plt.subplot(2,2,1)
            plt.plot(epochs, df["train_loss"], label={model_name})
            plt.title("Train Loss")
            plt.xlabel("Epochs")
            plt.legend()
            plt.savefig(tmpfile, format="png")
            tmpfile.seek(0)
            encoded = base64.b64encode(tmpfile.read()).decode("utf-8")
            train_loss.path = f"{train_loss.path}.html"
            html = f"<img src='data:image/png;base64,{encoded}'>"
            with open(train_loss.path, 'w') as f:
                f.write(html)

                
                
            # Save the model with help from utils.py on the local PVC and if export_bucket is provided then on minio storage also.
            utils.save_model(model=model,
                             model_dir=MODEL_DIR,
                             model_name=MODEL_NAME + ".pth",
                             export_bucket=EXPORT_BUCKET)

            print("saving to kfp now")
        
            # Saving the model as kfp output artifcat. 
    
            torch.save(model.state_dict(),
                       model_artifact_path)

            # Below steps logs the parameters to be used in the next step to be used by mlflow server
            with open(parameters_json_path, 'w') as f:
                json.dump({'lr': learning_rate, 'batch_size': batch_size, 'epochs': num_epochs}, f)
        
            # return the dictionary , 
            return result

Overwriting src/components/model_train_cnn/model_train_component.py


In [104]:
!kfp component build src/components/model_inference --component-filepattern model_inference_component.py --no-push-image

Building component using KFP package path: kfp==2.7.0
Found 1 component(s) in file /Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_inference/model_inference_component.py:
Model inference: ComponentInfo(name='Model inference', function_name='model_inference', func=<function model_inference at 0x11035e830>, target_image='mohitverma1688/model_inference_component:v0.20', module_path=PosixPath('/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_inference/model_inference_component.py'), component_spec=ComponentSpec(name='model-inference', implementation=Implementation(container=ContainerSpecImplementation(image='mohitverma1688/model_inference_component:v0.20', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet --no-w

In [103]:
%%writefile src/components/model_inference/model_inference_component.py

# Creating a python script to predict a model

from typing import Dict, NamedTuple
from kfp import dsl
from kfp.dsl import Dataset,Output,Artifact,OutputPath,InputPath, Model,Input,HTML

@dsl.component(base_image='mohitverma1688/model_train_component:v0.1',
               target_image='mohitverma1688/model_inference_component:v0.20',
               packages_to_install=['pandas','matplotlib']
               )

def model_inference(model_artifact_path: InputPath('Model'),
                    num_epochs: int,
                    batch_size:int,
                    learning_rate: float,
                    train_dir: str,
                    test_dir: str,
                    model_name: str,
                    test_loss: Output[HTML],
                    metrics_json_path: OutputPath('Artifact')
                    ) -> Dict[str, str]:

    import torch
    import model_builder, utils, model_inference, data_setup
    import json
    import os
    import pandas as pd
    from torchvision import transforms
    from io import BytesIO
    import base64
    import matplotlib.pyplot as plt
    

    # Setup hyperparameters
    NUM_EPOCHS = num_epochs
    BATCH_SIZE = batch_size
    LEARNING_RATE = learning_rate

    # Setup directories
    TRAIN_DIR = train_dir
    TEST_DIR = test_dir


    # Create transforms
    data_transform = transforms.Compose([
      transforms.Resize((64, 64)),
      transforms.ToTensor()
    ])
    # Create DataLoaders with help from data_setup.py
    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=TRAIN_DIR,
        test_dir=TEST_DIR,
        transform=data_transform,
        batch_size=BATCH_SIZE
    )  
    
    # Set up device   
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load the model
    model = model_builder.TinyVGG(input_shape=3,
                                  hidden_units=10,
                                  output_shape=3).to(device)
    
    model.load_state_dict(torch.load(model_artifact_path))

    # Set loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=LEARNING_RATE)

    # Start training with help from engine.py
    test_acc_last_epoch,test_loss_last_epoch,results = model_inference.test_result(model=model,
                       test_dataloader=test_dataloader,
                       loss_fn=loss_fn,
                       epochs=NUM_EPOCHS,
                       device=device)
    
    df = pd.DataFrame(results)
    #Get number of epochs

    epochs = range(len(df))
        

    # Plot train loss
    tmpfile = BytesIO()
    plt.figure(figsize=(15,10))
    plt.subplot(2,2,1)
    plt.plot(epochs, df["test_loss"], label={model_name})
    plt.title("Test Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.savefig(tmpfile, format="png")
    tmpfile.seek(0)
    encoded = base64.b64encode(tmpfile.read()).decode("utf-8")
    test_loss.path = f"{test_loss.path}.html"
    html = f"<img src='data:image/png;base64,{encoded}'>"
    with open(test_loss.path, 'w') as f:
        f.write(html)
    

    # Below output parameters will be used by the mlflow to log the model in the next step.

    with open(metrics_json_path, 'w') as f:
        json.dump({'accuracy': test_acc_last_epoch, 'loss': test_loss_last_epoch}, f)
        
    return {"model_name" : model_name, 
            "test_acc_last_epoch": test_acc_last_epoch,
            "test_loss_last_epoch": test_loss_last_epoch}


Overwriting src/components/model_inference/model_inference_component.py


In [90]:
!kfp component build src/components/register_model --component-filepattern register_model_component.py --no-push-image

Building component using KFP package path: kfp==2.7.0
Found 1 component(s) in file /Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/register_model/register_model_component.py:
Register model: ComponentInfo(name='Register model', function_name='register_model', func=<function register_model at 0x112536830>, target_image='mohitverma1688/register_model_component:v0.30', module_path=PosixPath('/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/register_model/register_model_component.py'), component_spec=ComponentSpec(name='register-model', implementation=Implementation(container=ContainerSpecImplementation(image='mohitverma1688/register_model_component:v0.30', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet --no-warn-script-

In [89]:
%%writefile src/components/register_model/register_model_component.py
 
#Creating a python script to register the model in MLflow

from typing import Dict, NamedTuple
from kfp import dsl
from kfp.dsl import Dataset,Output,Artifact,OutputPath,InputPath, Model,Input

@dsl.component(base_image='mohitverma1688/model_train_component:v0.1',
               target_image='mohitverma1688/register_model_component:v0.30',
               packages_to_install=['mlflow','GitPython','numpy']
               )
def register_model(parameters_json_path: InputPath('Artifact'),
                   metrics_json_path: InputPath('Artifact'),
                   model_artifact_path: InputPath('Model'),
                   experiment_name: str,
                   aws_access_key_id: str,
                   aws_secret_access_key: str,
                  ) -> dict:
    
    import mlflow
    import torch
    import json
    import model_builder
    import os
    from mlflow.types import Schema, TensorSpec
    import numpy as np
    from mlflow.models import ModelSignature

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the model
    model = model_builder.TinyVGG(input_shape=3,
                                  hidden_units=10,
                                  output_shape=3).to(device)
    
    model.load_state_dict(torch.load(model_artifact_path))
    
    # Log the model artifact in the MlFlow artifacts .

    with open('/tmp/model_builder.py', 'w') as f:
        f.write('''
import torch
from torch import nn
class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion
            ''')

    # Load the parameters and metrics for mlflow to register. 
    with open(metrics_json_path) as f:
        metrics = json.load(f)
        print(metrics)
    with open(parameters_json_path) as f:
        parameters = json.load(f)
        print(parameters)

    #Define the mlflow tracking URI and experiment name
    tracking_uri = 'http://my-mlflow.kubeflow:5000'
    experiment_name = experiment_name
    mlflow.tracking.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment_name)
    reg_model_name = "CNN-TinyVG-Model"

    # Logs the enviornment variable to be used by minio
    os.environ["GIT_PYTHON_REFRESH"] = "quiet"
    os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio-service.kubeflow:9000"
    os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
    os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key

    #Define the model schema

    input_schema = Schema([TensorSpec(np.dtype(np.float32), (1, 3, 64,64))])
    output_schema = Schema([TensorSpec(np.dtype(np.float32), (1, 3))])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    

    # Star the mlflow server to register the model 
    with mlflow.start_run() as run:
        for metric in metrics:
            mlflow.log_metric(metric, metrics[metric])
        mlflow.log_params(parameters)

        artifact_path = "CNN-TinyVG"
        mlflow.pytorch.log_model(model,
                                 registered_model_name=reg_model_name,
                                 signature=signature,
                                 artifact_path=artifact_path,
                                code_paths=['/tmp/model_builder.py'])

        # Example snippet to log the scripted model 
        #scripted_pytorch_model = torch.jit.script(model)
        
        #mlflow.pytorch.log_model(scripted_pytorch_model, "scripted_model")
        
        run_id = mlflow.last_active_run().info.run_id
        print(f"Logged data and model in run {run_id}...")

        # Run time model uri 
        model_uri = f"runs:/{run_id}/CNN-model"
        
        #Another way of loading model at runtime
        #loaded_model = mlflow.pytorch.load_model(model_uri)
        
        return {"artifact_path": artifact_path, "artifact_uri": run.info.artifact_uri}
        

                 

Overwriting src/components/register_model/register_model_component.py


In [109]:
!kfp component build src/components/model_eval --component-filepattern model_eval_component.py --no-push-image

Building component using KFP package path: kfp==2.7.0
Found 1 component(s) in file /Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_eval/model_eval_component.py:
Predict on sample image: ComponentInfo(name='Predict on sample image', function_name='predict_on_sample_image', func=<function predict_on_sample_image at 0x111386830>, target_image='mohitverma1688/model_eval_component:v0.8', module_path=PosixPath('/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/python-containerized1/src/components/model_eval/model_eval_component.py'), component_spec=ComponentSpec(name='predict-on-sample-image', implementation=Implementation(container=ContainerSpecImplementation(image='mohitverma1688/model_eval_component:v0.8', command=['sh', '-c', '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quie

In [108]:
%%writefile src/components/model_eval/model_eval_component.py
from kfp import dsl
from typing import NamedTuple
from kfp.dsl import Dataset,Output,Artifact,OutputPath,InputPath, Model,Input

@dsl.component(base_image='mohitverma1688/model_train_component:v0.1',
               target_image='mohitverma1688/model_eval_component:v0.8',
               packages_to_install=['mlflow','GitPython']
               )
def predict_on_sample_image(test_dir: str, 
                         model_info: dict, 
                         image_path: str,
                         aws_access_key_id: str, 
                         aws_secret_access_key: str ,
                         ) -> NamedTuple('outputs', [('model_uri', str),('pred_label_class', str)]):
    import torch
    import torchvision
    import utils
    import os
    import mlflow
    import json

    # Set Minio credentials in the environment
    os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio-service.kubeflow:9000"
    os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
    os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key

    
    # Set up the class names, below code uses utis.py to get the class_names from the dataset
    
    DATA_DIR = test_dir 
    class_names = utils.class_names_found(DATA_DIR)
    #print(class_names)
     
    #class_names = ["pizza", "steak", "sushi"]
    
    # Set up device   
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    
    artifact_path = model_info["artifact_path"]
    artifact_uri = model_info["artifact_uri"]

    # Loading the model from the mlFlow artifact repository 
    
    mlflow.set_tracking_uri("http://my-mlflow.kubeflow:5000")
    model_uri = f"{artifact_uri}/{artifact_path}"
    model = mlflow.pytorch.load_model(model_uri)

    # Below section is to use the the random path if the image path is not provided. A random image from test data in dataset is selected for prediction. 
    # Make sure that the downloaded data has the path for testing some images for example "/data/predict/123.jpg"
    
    if image_path == None:
    
        test_data_paths =  list(Path("/data/test").glob("*/*.jpg"))
        image_path = random.choice(test_data_paths)
        print(image_path)

    # Load in the image and turn it into torch.float32 ( same type as model)
    image = torchvision.io.read_image(image_path).type(torch.float32)

    # Preprocess the image to get between 0 and 1
    image = image / 255.

    # Resize the image to be of same size as model.
    transform = torchvision.transforms.Resize(size=(64,64))
    image = transform(image)

    # Predict on image
    model.eval()
    with torch.inference_mode():
        # Put image on the model
        image = image.to(device)

        # Get the pred_logits
        pred_logits = model(image.unsqueeze(dim=0))  # Adding a new dimension for the batch size.

        # Get the pred probs
        pred_prob = torch.softmax(pred_logits, dim=1)

        # Get the pred_labels
        pred_label = torch.argmax(pred_prob, dim=1)
        pred_label_class = class_names[pred_label]

    print(f"[INFO] Pred class: {pred_label_class}, Pred_prob: {pred_prob.max():.3f}")
    pred_prob_max = pred_prob.max()
    print(type(pred_prob_max))

    outputs = NamedTuple("outputs", model_uri=str, pred_label_class=str)
    
    return outputs(model_uri, pred_label_class)

Overwriting src/components/model_eval/model_eval_component.py


In [39]:
%%writefile src/components/model_deployment/model_deployment_component.py
from kfp import dsl
@dsl.component(
    packages_to_install=["kserve==0.12.0","ray[serve]<=2.9.3,>=2.9.2"],
    base_image="python:3.10",
)
def model_serving(model_uri: str):
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1TorchServeSpec
    from kserve import V1beta1ModelSpec
    from kserve import V1beta1ModelFormat
    import os

    namespace = utils.get_default_target_namespace()
    
    name='cnn-tinyvg-v1'
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec( model=V1beta1ModelSpec(model_format=V1beta1ModelFormat(name='mlflow')),
                                   service_account_name='mlflow-sa',
                                   pytorch=(V1beta1TorchServeSpec(storage_uri=model_uri)))))
    KServe = KServeClient()
    KServe.create(isvc)
    


Overwriting src/components/model_deployment/model_deployment_component.py


In [129]:
%%writefile pipeline.py
from kfp import kubernetes
from kubernetes import client, config
import base64
from kfp import dsl
from kfp import compiler
from src.components.data_download.data_download_component import dataset_download
from src.components.model_train_cnn.model_train_component import model_train
from src.components.model_inference.model_inference_component import model_inference
from src.components.register_model.register_model_component import register_model
from src.components.model_eval.model_eval_component import predict_on_sample_image
#from src.components.model_deployment.model_deployment_component import model_serving


BASE_PATH="/data"
URL="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
#INPUT_BUCKET="datanewbucket"
NUM_EPOCHS=10
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
MODEL_NAME = "cnn_tinyvg_v1"
MODEL_DIR = "/data/models"
EXPORT_BUCKET = "modeloutput"
TRAIN_DIR = "/data/train"
TEST_DIR = "/data/test"
IMAGE_PATH = "/data/test/pizza/1152100.jpg"
EXPERIMENT_NAME = "CNN-TinyVG-Demo-exp1"

@dsl.pipeline(name='CNN-TinyVG-Demo',
              description='This pipeline is a demo for training,evaluating and deploying Convutional Neural network',
              display_name='Kubeflow-MlFLow-Demo')



def kubeflow_pipeline(base_path: str = BASE_PATH,
                     url:str = URL,
                     batch_size:int = BATCH_SIZE,
                     train_dir:str = TRAIN_DIR,
                     test_dir:str = TEST_DIR,
                     num_epochs: int = NUM_EPOCHS,
                     hidden_units:int = HIDDEN_UNITS,
                     learning_rate:float = LEARNING_RATE,
                     model_name: str = MODEL_NAME,
                     model_dir: str = MODEL_DIR,
                     export_bucket: str = EXPORT_BUCKET,
                     image_path: str = IMAGE_PATH,
                     experiment_name: str = EXPERIMENT_NAME
                     ):
    # Load Kubernetes configuration
    config.load_kube_config()

    # Fetch the Minio credentials from the secret

    secret_name = "minio-credentials"
    secret_namespace = "kubeflow"
    secret_key_id = "AWS_ACCESS_KEY_ID"
    secret_key_access = "AWS_SECRET_ACCESS_KEY"

    v1 = client.CoreV1Api()
    secret = v1.read_namespaced_secret(secret_name, namespace=secret_namespace)


    # Convert bytes to string
    aws_access_key_id = base64.b64decode(secret.data[secret_key_id]).decode('utf-8')
    aws_secret_access_key = base64.b64decode(secret.data[secret_key_access]).decode('utf-8')
   
    pvc1 = kubernetes.CreatePVC(
        # can also use pvc_name instead of pvc_name_suffix to use a pre-existing PVC
        pvc_name='kubeflow-pvc8',
        access_modes=['ReadWriteOnce'],
        size='500Mi',
        storage_class_name='standard',
    )
    
    task1 = dataset_download(base_path=base_path,
                            url=url,
                            )
    task1.set_caching_options(False)
    
    task2 = model_train(batch_size=batch_size,
                        num_epochs=num_epochs,
                        train_dir=train_dir,
                        test_dir=test_dir,
                        hidden_units=hidden_units,
                        learning_rate=learning_rate,
                        model_name=model_name,
                        model_dir=model_dir,
                        export_bucket=export_bucket,
                        ).after(task1)
    task2.set_caching_options(False)
    
    task3 = model_inference(test_dir=test_dir,
                           model_artifact_path=task2.outputs["model_artifact_path"],
                           train_dir=train_dir,
                           learning_rate=learning_rate,
                           batch_size=batch_size,
                           num_epochs=num_epochs,
                           model_name=model_name
                           ).after(task2)
    task3.set_caching_options(False)
    
    task4 = register_model(model_artifact_path=task2.outputs["model_artifact_path"],
                           parameters_json_path=task2.outputs["parameters_json_path"],
                           metrics_json_path=task3.outputs["metrics_json_path"],
                           aws_access_key_id=aws_access_key_id,
                           aws_secret_access_key=aws_secret_access_key,                         
                           experiment_name=experiment_name).after(task3)
    task4.set_caching_options(False)

    task5 = predict_on_sample_image(test_dir=test_dir,
                                 image_path=image_path,
                                 model_info=task4.output,
                                 aws_access_key_id=aws_access_key_id,
                                 aws_secret_access_key=aws_secret_access_key).after(task4)
    task5.set_caching_options(False)

    

#    task6 = model_serving(model_uri=task5.outputs['model_uri']).after(task5)
#    task6.set_caching_options(False)
    
              
                                                  
    kubernetes.mount_pvc(
        task1,
        pvc_name=pvc1.outputs['name'],
        mount_path='/data',
    )
    kubernetes.mount_pvc(
        task2,
        pvc_name=pvc1.outputs['name'],
        mount_path='/data',
    )
    kubernetes.mount_pvc(
        task3,
        pvc_name=pvc1.outputs['name'],
        mount_path='/data',
    )
    kubernetes.mount_pvc(
        task4,
        pvc_name=pvc1.outputs['name'],
        mount_path='/data',
    )
    kubernetes.mount_pvc(
        task5,
        pvc_name=pvc1.outputs['name'],
        mount_path='/data',
    )
#    kubernetes.mount_pvc(
#        task6,
#        pvc_name=pvc1.outputs['name'],
#        mount_path='/data',
#    )
    delete_pvc1 = kubernetes.DeletePVC(pvc_name=pvc1.outputs['name']).after(task5)

compiler.Compiler().compile(kubeflow_pipeline, 'kubeflow-demo2.yaml')

Overwriting pipeline.py


In [6]:
from kfp import dsl
deploy_step = dsl.ResourceOp(
        name="seldondeploy",
        k8s_resource=seldon_config,
        attribute_outputs={"name": "{.metadata.name}"})

AttributeError: module 'kfp.dsl' has no attribute 'ResourceOp'

In [25]:
!pip install ray[serve]<=2.9.3,>=2.9.2

zsh:1: no matches found: ray[serve]


In [21]:
import kserve

ImportError: cannot import name 'RayServeHandle' from 'ray.serve.handle' (/Users/mverma/Documents/STUDY-Mac/data-science/sample_project/env/lib/python3.10/site-packages/ray/serve/handle.py)

In [ ]:
%%writefile run.py
from kfp.client import Client

client = Client(host='http://localhost:8002')
run = client.create_run_from_pipeline_package(
    'kubeflow-demo2.yaml',
     )

In [ ]:
x = {'train_loss': [1.1130876988172531, 1.1088187843561172, 1.0958865135908127, 1.0962762534618378, 1.0938437581062317, 1.1108780950307846, 1.0907627046108246, 1.1078475266695023, 1.1044471561908722, 1.09113809466362], 'train_acc': [0.2890625, 0.29296875, 0.4140625, 0.29296875, 0.41796875, 0.3046875, 0.42578125, 0.3046875, 0.3046875, 0.42578125], 'test_loss': [1.094068129857381, 1.107581655184428, 1.1117815176645915, 1.1201920906702678, 1.1264954408009846, 1.128881613413493, 1.1216977437337239, 1.1192431847254436, 1.1100176175435383, 1.1010971864064534], 'test_acc': [0.19791666666666666, 0.19791666666666666, 0.19791666666666666, 0.19791666666666666, 0.2604166666666667, 0.2604166666666667, 0.2604166666666667, 0.2604166666666667, 0.2604166666666667, 0.2604166666666667]}

In [ ]:
for key,value in x.items():
    results = { }
    value = value[9]
    result[key] = value
result

    
    

In [ ]:
from kfp import dsl
from kfp.dsl import Dataset
from kfp.dsl import Input

@dsl.component
def print_artifact_properties(dataset: Input[Dataset]):
    with open(dataset.path) as f:
        lines = f.readlines()
    
    print('Information about the artifact')
    print('Name:', dataset.name)
    print('URI:', dataset.uri)
    print('Path:', dataset.path)
    print('Metadata:', dataset.metadata)
    
    return len(lines)

In [ ]:
NUM_EPOCH = 5
NUM_EPOCH= NUM_EPOCH - 1
NUM_EPOCH

In [ ]:
import json

In [ ]:
numbers = [2, 3, 4, 0, 42, 17]
numbers = numbers.sort()
print(numbers)

In [ ]:
-> Dict[str, list] 

In [ ]:
!pip uninstall kserve -y

In [ ]:
from kfp import dsl
from kfp.dsl import Dataset
from kfp.dsl import Input

@dsl.component
def print_artifact_properties(dataset: Input[Dataset]):
    with open(dataset.path) as f:
        lines = f.readlines()
    
    print('Information about the artifact')
    print('Name:', dataset.name)
    print('URI:', dataset.uri)
    print('Path:', dataset.path)
    print('Metadata:', dataset.metadata)
    
    return len(lines)

In [ ]:
!pip list | grep ray

In [7]:
!pip install ray[serve]<=2.9.3,>=2.9.2

zsh:1: no matches found: ray[serve]


In [111]:
import os
from io import BytesIO
import PIL.Image as Image
import base64

import numpy as np
import torch
import mlflow

class Inference(object):
    def __init__(self):
        self.device = 'cpu'
        mlflow.set_tracking_uri("http://20.28.195.42/mlflow/")
        mlflow.set_registry_uri("http://20.28.195.42/mlflow/")
        self.model_path = f"runs:/{os.environ['RUN_ID']}/models"
        self.model = mlflow.pytorch.load_model(self.model_path)
        print(self.model)

    def predict(self, X, feature_names):
        X = X[0].encode()
        im_bytes = base64.b64decode(X)
        im_file = BytesIO(im_bytes)
        img = Image.open(im_file)
        img = np.asarray(img.resize((128, 128))).astype(np.float32) / 255.0
        if img.ndim == 2:
            img = np.stack([img, img, img], axis=-1)
        img = np.transpose(img, (-1, 0, 1))
        img = np.expand_dims(img, axis=0)
        img = torch.from_numpy(img).to(self.device)
        op = torch.nn.Sigmoid()(self.model(img))
        op = torch.where(op > 0.5, 1, 0).item()
        return [op]


if __name__ == "__main__":
    img = Image.open("/data/test/pizza/1152100.jpg")
    im_file = BytesIO()
    img.save(im_file, format="JPEG")
    im_bytes = im_file.getvalue()  # im_bytes: image in binary format.
    im_b64 = base64.b64encode(im_bytes)
    im_b64 = im_b64.decode()
    op = Inference().predict(im_b64, ["Image"])
    print(op)

FileNotFoundError: [Errno 2] No such file or directory: '/data/test/pizza/1152100.jpg'

In [112]:
!pip list | grep kfp

kfp                                2.7.0
kfp-kubernetes                     1.2.0
kfp-pipeline-spec                  0.3.0
kfp-server-api                     2.0.5
